In [9]:
%cd practicas/

[Errno 2] No such file or directory: 'practicas/'
/workspace/practicas


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [10]:
from pyspark.sql import SparkSession

spark = ( SparkSession.builder
         .appName("pr502")
         .master("spark://spark-master:7077")
         .getOrCreate()
         )
 
sc = spark.sparkContext

# Dataset 1: Datos para la predicción del rendimiento en cultivos

In [11]:
from pyspark.sql.types import StructType, StructField, BooleanType, IntegerType, StringType, DoubleType, LongType, TimestampType
from pyspark.sql.functions import col


schema_crop = StructType([
    StructField("Crop", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("Soil_Type", StringType(), True),
    StructField("Soil_ph", DoubleType(), True),
    StructField("Rainfall_mm", DoubleType(), True),
    StructField("Temperature_c", DoubleType(), True),
    StructField("Humidity_pct", DoubleType(), True),
    StructField("Fertilizer_Used_kg", DoubleType(), True),
    StructField("Irrigation", StringType(), True),
    StructField("Pesticides_Used_kg", DoubleType(), True),
    StructField("Planting_Density", DoubleType(), True),
    StructField("Previous_Crop", StringType(), True),
    StructField("Yield_ton_per_ha", DoubleType(), True)
])

df_crop = (spark.read
             .format("csv")
             .schema(schema_crop)
             .option("header", "True")
             .load("./data/crop_yield_dataset.csv"))
df_crop.printSchema()
df_crop.show(5)
df_crop.count()

root
 |-- Crop: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Soil_Type: string (nullable = true)
 |-- Soil_ph: double (nullable = true)
 |-- Rainfall_mm: double (nullable = true)
 |-- Temperature_c: double (nullable = true)
 |-- Humidity_pct: double (nullable = true)
 |-- Fertilizer_Used_kg: double (nullable = true)
 |-- Irrigation: string (nullable = true)
 |-- Pesticides_Used_kg: double (nullable = true)
 |-- Planting_Density: double (nullable = true)
 |-- Previous_Crop: string (nullable = true)
 |-- Yield_ton_per_ha: double (nullable = true)

+------+--------+---------+-------+-----------+-------------+------------+------------------+----------+------------------+----------------+-------------+----------------+
|  Crop|  Region|Soil_Type|Soil_ph|Rainfall_mm|Temperature_c|Humidity_pct|Fertilizer_Used_kg|Irrigation|Pesticides_Used_kg|Planting_Density|Previous_Crop|Yield_ton_per_ha|
+------+--------+---------+-------+-----------+-------------+------------+-------

10000

## 1. Selección de características

In [12]:
df_sel = df_crop.select("Crop", "Region", "Temperature_C", "Rainfall_mm", "Irrigation", "Yield_ton_per_ha")
df_sel.show(5)

+------+--------+-------------+-----------+----------+----------------+
|  Crop|  Region|Temperature_C|Rainfall_mm|Irrigation|Yield_ton_per_ha|
+------+--------+-------------+-----------+----------+----------------+
| Maize|Region_C|         19.7|     1485.4|      Drip|          101.48|
|Barley|Region_D|         29.1|      399.4| Sprinkler|          127.39|
|  Rice|Region_C|         30.5|      980.9| Sprinkler|           68.99|
| Maize|Region_D|         26.4|     1054.3|      Drip|          169.06|
| Maize|Region_D|         20.4|      744.6|      Drip|          118.71|
+------+--------+-------------+-----------+----------+----------------+
only showing top 5 rows



## 2. Normalización de nombre

In [13]:
df_renamed = ( df_sel
    .withColumnRenamed("Temperature_C", "Temperatura")
    .withColumnRenamed("Rainfall_mm", "Lluvia")
    .withColumnRenamed("Yield_ton_per_ha", "Rendimiento")
 )
df_renamed.show(5)


+------+--------+-----------+------+----------+-----------+
|  Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+------+--------+-----------+------+----------+-----------+
| Maize|Region_C|       19.7|1485.4|      Drip|     101.48|
|Barley|Region_D|       29.1| 399.4| Sprinkler|     127.39|
|  Rice|Region_C|       30.5| 980.9| Sprinkler|      68.99|
| Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
| Maize|Region_D|       20.4| 744.6|      Drip|     118.71|
+------+--------+-----------+------+----------+-----------+
only showing top 5 rows



## 3. Filtrado de datos

In [14]:
df_renamed.filter( (col("Crop") == "Maize") & (col("Temperatura") > 25)).show(5)

+-----+--------+-----------+------+----------+-----------+
| Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+-----+--------+-----------+------+----------+-----------+
|Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
|Maize|Region_C|       32.4| 846.1|      None|      162.2|
|Maize|Region_A|       26.6| 362.5| Sprinkler|      95.23|
|Maize|Region_C|       33.7|1193.3|      None|     110.57|
|Maize|Region_C|       27.8| 695.2|     Flood|     143.84|
+-----+--------+-----------+------+----------+-----------+
only showing top 5 rows



## 4. Encadenamiento

In [15]:
df_sel = df_crop \
    .select("Crop", "Region", "Temperature_C", "Rainfall_mm", "Irrigation", "Yield_ton_per_ha") \
    .withColumnRenamed("Temperature_C", "Temperatura") \
    .withColumnRenamed("Rainfall_mm", "Lluvia") \
    .withColumnRenamed("Yield_ton_per_ha", "Rendimiento") \
    .filter( (col("Crop") == "Maize") & (col("Temperatura") > 25)).show(5)

+-----+--------+-----------+------+----------+-----------+
| Crop|  Region|Temperatura|Lluvia|Irrigation|Rendimiento|
+-----+--------+-----------+------+----------+-----------+
|Maize|Region_D|       26.4|1054.3|      Drip|     169.06|
|Maize|Region_C|       32.4| 846.1|      None|      162.2|
|Maize|Region_A|       26.6| 362.5| Sprinkler|      95.23|
|Maize|Region_C|       33.7|1193.3|      None|     110.57|
|Maize|Region_C|       27.8| 695.2|     Flood|     143.84|
+-----+--------+-----------+------+----------+-----------+
only showing top 5 rows



# Dataset 2: Lugares famosos del mundo

In [16]:
schema_world = StructType([
    StructField("Place_Name", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Annual_Visitors_Millions", DoubleType(), True),
    StructField("Type", StringType(), True),
    StructField("UNESCO_World_Heritage", StringType(), True),
    StructField("Year_Built", StringType(), True),
    StructField("Entry_Fee_USD", IntegerType(), True),
    StructField("Best_Visit_Month", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("Tourism_Revenue_Million_USD", StringType(), True),
    StructField("Average_Visit_Duration_Hours", LongType(), True),
    StructField("Famous_For", StringType(), True),
])

df_city = (spark.read
             .format("csv")
             .schema(schema_world)
             .option("header", "True")
             .load("./data/world_famous_places_2024.csv"))
df_city.printSchema()
df_city.show(5)

root
 |-- Place_Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Annual_Visitors_Millions: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- UNESCO_World_Heritage: string (nullable = true)
 |-- Year_Built: string (nullable = true)
 |-- Entry_Fee_USD: integer (nullable = true)
 |-- Best_Visit_Month: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Tourism_Revenue_Million_USD: string (nullable = true)
 |-- Average_Visit_Duration_Hours: long (nullable = true)
 |-- Famous_For: string (nullable = true)

+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+
|         Place_Name|      Country|            City|Annual_Visitors_Millions|              Type|UNESCO_World_Heritage|      Year_Built|Ent

## 1. Selección de datos críticos

In [17]:
df_base = df_city.select("Place_Name", "Country", "UNESCO_World_Heritage", "Entry_Fee_USD", "Annual_Visitors_Millions")
df_base.show(5)

+-------------------+-------------+---------------------+-------------+------------------------+
|         Place_Name|      Country|UNESCO_World_Heritage|Entry_Fee_USD|Annual_Visitors_Millions|
+-------------------+-------------+---------------------+-------------+------------------------+
|       Eiffel Tower|       France|                   No|           35|                     7.0|
|       Times Square|United States|                   No|            0|                    50.0|
|      Louvre Museum|       France|                  Yes|           22|                     8.7|
|Great Wall of China|        China|                  Yes|           10|                    10.0|
|          Taj Mahal|        India|                  Yes|           15|                     7.5|
+-------------------+-------------+---------------------+-------------+------------------------+
only showing top 5 rows



## 2. Traducción y simplificación

In [18]:
df_se = ( df_base
   .withColumnRenamed("Place_Name", "Lugar")
   .withColumnRenamed("UNESCO_World_Heritage", "Es_UNESCO")
   .withColumnRenamed("Entry_Fee_USD", "Precio_Entrada")
   .withColumnRenamed("Annual_Visitors_Millions", "Visitantes_Millones")
)

df_se.show(5)

+-------------------+-------------+---------+--------------+-------------------+
|              Lugar|      Country|Es_UNESCO|Precio_Entrada|Visitantes_Millones|
+-------------------+-------------+---------+--------------+-------------------+
|       Eiffel Tower|       France|       No|            35|                7.0|
|       Times Square|United States|       No|             0|               50.0|
|      Louvre Museum|       France|      Yes|            22|                8.7|
|Great Wall of China|        China|      Yes|            10|               10.0|
|          Taj Mahal|        India|      Yes|            15|                7.5|
+-------------------+-------------+---------+--------------+-------------------+
only showing top 5 rows



## 3. Filtrado

In [19]:
df_se.filter( (col("Es_UNESCO").contains("Yes")) & (col("Precio_Entrada") <= 20)).show()

+--------------------+--------------+---------+--------------+-------------------+
|               Lugar|       Country|Es_UNESCO|Precio_Entrada|Visitantes_Millones|
+--------------------+--------------+---------+--------------+-------------------+
| Great Wall of China|         China|      Yes|            10|               10.0|
|           Taj Mahal|         India|      Yes|            15|                7.5|
|           Colosseum|         Italy|      Yes|            18|               7.65|
|      Forbidden City|         China|      Yes|             8|                9.0|
|Notre-Dame Cathedral|        France|      Yes|             0|               13.0|
|Great Pyramid of ...|         Egypt|      Yes|            20|                2.8|
|Leaning Tower of ...|         Italy|      Yes|            20|                5.0|
|           Acropolis|        Greece|      Yes|            13|                4.0|
|             Big Ben|United Kingdom|      Yes|             0|                5.5|
+---

# Dataset 3: Registro turídtico de Castilla y León

In [20]:
schema_cyl = StructType([
    StructField("establecimiento", StringType(), True),
    StructField("n_registro", StringType(), True),
    StructField("codigo", StringType(), True),
    StructField("tipo", StringType(), True),
    StructField("categoria", StringType(), True),
    StructField("especialidades", StringType(), True),
    StructField("clase", StringType(), True),
    StructField("nombre", StringType(), True),
    StructField("direccion", StringType(), True),
    StructField("c_postal", StringType(), True),
    StructField("provincia", StringType(), True),
    StructField("municipio", StringType(), True),
    StructField("localidad", StringType(), True),
    StructField("nucleo", StringType(), True),
    StructField("telefono_1", StringType(), True),
    StructField("telefono_2", StringType(), True),
    StructField("telefono_3", StringType(), True),
    StructField("email", StringType(), True),
    StructField("web", StringType(), True),
    StructField("q_calidad", StringType(), True),
    StructField("posada_real", StringType(), True),
    StructField("plazas", IntegerType(), True),
    StructField("gps_longitud", DoubleType(), True),
    StructField("gps_latitud", DoubleType(), True),
    StructField("accesible_a_personas_con_discapacidad", StringType(), True),
    StructField("column_27", StringType(), True),
    StructField("posicion", StringType(), True)
])

df_cyl = (spark.read
             .format("csv")
             .schema(schema_cyl)
             .option("header", "True")
             .option("sep", ";")
             .load("./data/registro-de-turismo-de-castilla-y-leon.csv"))
df_cyl.printSchema()
df_cyl.show(5)

root
 |-- establecimiento: string (nullable = true)
 |-- n_registro: string (nullable = true)
 |-- codigo: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- especialidades: string (nullable = true)
 |-- clase: string (nullable = true)
 |-- nombre: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- c_postal: string (nullable = true)
 |-- provincia: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- localidad: string (nullable = true)
 |-- nucleo: string (nullable = true)
 |-- telefono_1: string (nullable = true)
 |-- telefono_2: string (nullable = true)
 |-- telefono_3: string (nullable = true)
 |-- email: string (nullable = true)
 |-- web: string (nullable = true)
 |-- q_calidad: string (nullable = true)
 |-- posada_real: string (nullable = true)
 |-- plazas: integer (nullable = true)
 |-- gps_longitud: double (nullable = true)
 |-- gps_latitud: double (nullable = true)
 |-- accesible_a_per

26/01/22 10:03:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 1. Selección y saneamiento

In [21]:
df_contactos = df_cyl.select("Nombre", "Tipo", "Provincia", "web", "Email")
df_contactos.show(5)

+--------------------+--------------------+---------+--------------------+--------------------+
|              Nombre|                Tipo|Provincia|                 web|               Email|
+--------------------+--------------------+---------+--------------------+--------------------+
|BERNARDO MORO MEN...|Profesional de Tu...| Asturias|                NULL|bernardomoro@hotm...|
|        LA SASTRERÍA|Casa Rural de Alq...|    Ávila|www.lasastreriade...|                NULL|
|         LAS HAZANAS|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
| LA CASITA DEL PAJAR|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
|            MARACANA|                 Bar|    Ávila|                NULL|emo123anatoliev@g...|
+--------------------+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



## 2. Renombreado estándar

In [22]:
df_limpio = ( df_contactos
                .withColumnRenamed("Nombre", "nombre_establecimiento")
                .withColumnRenamed("Tipo", "categoria_actividad")
                .withColumnRenamed("web", "sitio_web")
                .withColumnRenamed("Email", "correo_electronico")
)
df_limpio.show(5)

+----------------------+--------------------+---------+--------------------+--------------------+
|nombre_establecimiento| categoria_actividad|Provincia|           sitio_web|  correo_electronico|
+----------------------+--------------------+---------+--------------------+--------------------+
|  BERNARDO MORO MEN...|Profesional de Tu...| Asturias|                NULL|bernardomoro@hotm...|
|          LA SASTRERÍA|Casa Rural de Alq...|    Ávila|www.lasastreriade...|                NULL|
|           LAS HAZANAS|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
|   LA CASITA DEL PAJAR|Casa Rural de Alq...|    Ávila|                NULL|lashazanas@hotmai...|
|              MARACANA|                 Bar|    Ávila|                NULL|emo123anatoliev@g...|
+----------------------+--------------------+---------+--------------------+--------------------+
only showing top 5 rows



## 3. Filtrado de texto

In [23]:
df_final = df_limpio.filter( (col("Provincia") == "Burgos") & (col("categoria_actividad").contains("Bodegas")) & (col("sitio_web").isNotNull()))
df_final.show()

+----------------------+--------------------+---------+--------------------+--------------------+
|nombre_establecimiento| categoria_actividad|Provincia|           sitio_web|  correo_electronico|
+----------------------+--------------------+---------+--------------------+--------------------+
|        BODEGAS TARSUS|g - Bodegas y los...|   Burgos|  www.tarsusvino.com|                NULL|
|  BODEGAS DOMINIO D...|g - Bodegas y los...|   Burgos|www.dominiodecair...|bodegas@dominiode...|
|    TERRITORIO LUTHIER|g - Bodegas y los...|   Burgos|territorioluthier...|luthier@territori...|
|    BODEGA COVARRUBIAS|g - Bodegas y los...|   Burgos| http://valdable.com|   info@valdable.com|
|  BODEGAS PASCUAL, ...|g - Bodegas y los...|   Burgos|222.bodegaspascua...|export@bodegaspas...|
|   BODEGAS VINUM VITAE|g - Bodegas y los...|   Burgos|      www.avañate.es|vinum.vitae.bodeg...|
|  VIÑEDOS Y BODEGAS...|g - Bodegas y los...|   Burgos|     www.ferratus.es|administracion@fe...|
|  BODEGAS Y VIÑEDOS